In [1]:
import numpy as np
import os
import json
from random import shuffle
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Normalization
from sklearn.model_selection import train_test_split


In [2]:
# Load all the data from the json files
dir = r"src/dataFiveNotePatterns/"
positive_note_patterns = []
negative_note_patterns = []

for file in os.listdir(dir):
    with open(dir + file, "r", encoding='utf-8') as f:
        data = json.load(f)
        positive_note_patterns += data["five_notes_patterns"]
        negative_note_patterns += data["bad_patterns"]
        


print("Positive note patterns: ", len(positive_note_patterns))
print("Negative note patterns: ", len(negative_note_patterns))


Positive note patterns:  3580
Negative note patterns:  1782


In [3]:
## make sure the two lists have the same length
if(len(positive_note_patterns) > len(negative_note_patterns)):
    positive_note_patterns = positive_note_patterns[:len(negative_note_patterns)]
else:
    negative_note_patterns = negative_note_patterns[:len(positive_note_patterns)]

print("Positive note patterns: ", len(positive_note_patterns))
print("Negative note patterns: ", len(negative_note_patterns))
all_note_patterns = positive_note_patterns + negative_note_patterns

print(all_note_patterns)
# shuffle the data
shuffle(all_note_patterns)


Positive note patterns:  1782
Negative note patterns:  1782
[[[6, 2, 39360], [6, 2, 39840], [5, 4, 40320], [6, 2, 40800], [5, 5, 41280], [1]], [[6, 2, 41760], [5, 4, 42240], [6, 2, 42720], [5, 2, 43200], [6, 5, 43680], [1]], [[6, 4, 44160], [6, 5, 44640], [5, 2, 45120], [6, 5, 45600], [6, 4, 46080], [1]], [[6, 0, 46560], [6, 2, 47040], [6, 2, 47520], [5, 4, 48000], [6, 2, 48480], [1]], [[5, 5, 48960], [6, 2, 49440], [5, 4, 49920], [6, 2, 50400], [5, 2, 50880], [1]], [[6, 5, 51360], [6, 4, 51840], [6, 5, 52320], [5, 2, 52800], [6, 5, 53280], [1]], [[6, 4, 53760], [6, 0, 54240], [6, 2, 54720], [6, 2, 55200], [5, 4, 55680], [1]], [[6, 2, 56160], [5, 5, 56640], [6, 2, 57120], [5, 4, 57600], [6, 2, 58080], [1]], [[5, 2, 58560], [6, 5, 59040], [6, 4, 59520], [6, 5, 60000], [5, 2, 60480], [1]], [[6, 5, 60960], [6, 4, 61440], [6, 0, 61920], [6, 2, 62400], [6, 2, 62880], [1]], [[5, 4, 63360], [6, 2, 63840], [5, 5, 64320], [6, 2, 64800], [5, 4, 65280], [1]], [[3, 4, 84000], [3, 2, 84160], [3, 0,

In [4]:
all_fret_patterns = []
all_fret_patterns_labels = []
for pattern in all_note_patterns:
    all_fret_patterns.append([[string, fret] for string, fret, _ in pattern[:-1]])
    all_fret_patterns_labels.append(pattern[-1])

print(all_fret_patterns)
all_fret_patterns = np.array(all_fret_patterns)
all_fret_patterns_labels = np.array(all_fret_patterns_labels)

# Reshape the data to fit the model input
all_fret_patterns = all_fret_patterns.reshape((all_fret_patterns.shape[0], -1))

# split the data into training and testing
train_ratio = 0.8
X_train, X_test, y_train, y_test = train_test_split(all_fret_patterns, all_fret_patterns_labels, test_size=1-train_ratio)
for x, y in zip(X_train, y_train):
    print(x, y)

[[[5, 3], [6, 4], [5, 2], [6, 3], [6, 1]], [[5, 2], [5, 2], [5, 2], [5, 4], [6, 5]], [[6, 3], [5, 3], [6, 4], [5, 2], [6, 3]], [[4, 8], [4, 7], [3, 9], [5, 7], [5, 0]], [[6, 0], [6, 0], [6, 0], [5, 9], [5, 10]], [[6, 2], [6, 2], [6, 2], [6, 2], [6, 2]], [[1, 10], [2, 0], [1, 0], [2, 0], [3, 0]], [[5, 14], [5, 15], [4, 12], [4, 14], [4, 15]], [[2, 7], [2, 10], [2, 7], [4, 0], [2, 7]], [[3, 7], [3, 7], [3, 7], [3, 6], [3, 6]], [[2, 17], [1, 15], [4, 7], [2, 17], [2, 17]], [[6, 0], [4, 4], [3, 2], [5, 3], [5, 2]], [[6, 0], [6, 0], [6, 0], [4, 9], [6, 0]], [[4, 4], [1, 14], [2, 15], [3, 16], [1, 14]], [[2, 18], [6, 5], [6, 5], [6, 5], [6, 5]], [[5, 7], [6, 0], [6, 0], [5, 7], [4, 5]], [[1, 10], [2, 0], [1, 3], [3, 0], [1, 0]], [[5, 2], [5, 2], [5, 2], [5, 2], [5, 2]], [[5, 2], [5, 2], [5, 2], [5, 2], [5, 2]], [[4, 5], [3, 11], [2, 14], [2, 15], [1, 14]], [[1, 15], [1, 14], [1, 15], [1, 14], [2, 17]], [[2, 15], [4, 5], [2, 15], [2, 0], [2, 14]], [[5, 2], [6, 3], [6, 1], [5, 3], [5, 2]], [[6

In [5]:
#noramlize the data
normalizer = Normalization()
normalizer.adapt(X_train)
X_train = normalizer(X_train)
X_test = normalizer(X_test)

In [6]:
#build the model
model = keras.Sequential([
    normalizer,
    keras.layers.Dense(64, input_shape=(X_train.shape[1],), activation='tanh'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(1, activation='sigmoid'),
    #keras.layers.Dense(1, activation='softmax')
])

print("Model Summary", model.summary())

c:\Program Files\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (2851, 10)             │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21 (88.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21 (88.00 B)

Model Summary None


In [7]:
print(X_train)
print(y_train)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=64)

tf.Tensor(
[[ 0.48435277  0.18859439  0.45607492 ...  0.15943019  1.0697895
  -0.30919692]
 [ 1.0787953  -0.99796855  1.0580516  ... -1.0394199   1.0697895
  -0.992811  ]
 [ 1.0787953  -0.99796855 -0.7478784  ... -1.0394199   1.0697895
  -0.992811  ]
 ...
 [-1.8934172   0.6971214  -1.3498551  ... -1.0394199  -1.9577765
  -0.992811  ]
 [ 0.48435277 -0.65895057  0.45607492 ... -0.69689125 -1.3522633
   1.7416453 ]
 [ 1.0787953  -0.48944157  1.0580516  ...  0.6732231  -1.3522633
  -0.992811  ]], shape=(2851, 10), dtype=float32)
[[1]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
Epoch 1/200


c:\Program Files\Python312\Lib\site-packages\keras\src\backend\tensorflow\nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5259 - loss: 0.6979
Epoch 2/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5085 - loss: 0.6944
Epoch 3/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5736 - loss: 0.6732
Epoch 4/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6142 - loss: 0.6560
Epoch 5/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6595 - loss: 0.6272
Epoch 6/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7351 - loss: 0.5736
Epoch 7/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8000 - loss: 0.5045
Epoch 8/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8304 - loss: 0.4347
Epoch 9/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8412 - loss: 0.4074
Epoch 10/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8434 - loss: 0.3867
Epoch 11/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8510 - loss: 0.3627
Epoch 12/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8761 - lo

In [143]:
#save the model
model.save("src/models/model_with_frets_and_strings.h5")